<div class="text_center", style="font-family: 'Times New Roman', Times, serif;
                                 text-align: center;
                                 font-weight: normal;
                                 font-size: 18px;
                                 padding: 0px 100px 0px 0px;">
    <span style="font-size: 15px; font-style: italic;">
        Partial Differential Equations with Applications in Physics and Industry
    </span>
    <hr>
    <br>
    <br> 
    <span style="font-size: 40px;"> Wave equation for vibrating membrane </span>
    <br>
    <br>
    <br>
    Marcin Kostrzewa, &nbsp;Marcin Miśkiewicz
</div>

<div class="text", style="font-family: 'Times New Roman', Times, serif;
                          font-weight: normal;
                          text-align: justify;
                          font-size: 18px;
                          padding: 0px 100px 0px 0px;">
    <h3>Abstract</h3>
    <hr>
    Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>

<div class="text", style="font-family: 'Times New Roman', Times, serif;
                          font-weight: normal;
                          text-align: justify;
                          font-size: 18px;
                          padding: 0px 100px 0px 0px;">

$$\frac{\partial^2 u}{\partial t^2} = c^2\Delta u$$

$\text{for } (x, y) \in \Omega \subset \mathbb{R}^d, \; t \in [0, T]$, where $ c $ is a positive constant related to the speed of wave propagation in a given medium, and
$$\Delta u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}.$$

</div>

<div class="text", style="font-family: 'Times New Roman', Times, serif;
                          font-weight: normal;
                          text-align: justify;
                          font-size: 18px;
                          padding: 0px 100px 0px 0px;">
    <h3>Results</h3>
    <hr>
    Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum. <a href=#1>[1]</a>
</div>

In [1]:
using Plots, Measures, LinearAlgebra, Logging, LaTeXStrings
Logging.disable_logging(Logging.Info); # disable ugly messages after generating animations

# plotting constants
COLORMAP_1 = cgrad([:midnightblue, :purple, :purple, :skyblue, :cyan, :cyan, :white])
SIZE_1 = (1000, 500);

In [2]:
function grid(; Δx::Float64, Δy::Float64, Ω::Dict) 
    A, B, C, D = Ω["A"], Ω["B"], Ω["C"], Ω["D"]
    xs = A:Δx:B
    ys = C:Δy:D
    nx, ny = length(xs), length(ys)
    xx = xs' .* ones(ny)
    yy = ys  .* ones(nx)'
    return (nx, ny, xx, yy)
end

grid (generic function with 1 method)

In [3]:
function explicit_finite_difference(; Δx::Float64, 
                                      Δy::Float64, 
                                      Δt::Float64, 
                                      c::Float64, 
                                      N::Int64,
                                      ICs::Dict, 
                                      BCs::Dict, 
                                      grid_params::Tuple)

    (nx, ny, xx, yy) = grid_params
    i = collect(2:nx-1)
    j = collect(2:ny-1)
    
    u_Ay, u_By = BCs["u(A, y, t)"], BCs["u(B, y, t)"]
    u_xC, u_xD = BCs["u(x, C, t)"], BCs["u(x, D, t)"]
    u0,   u_t0 = ICs["u(x, y, 0)"], ICs["∂u/∂t(x, y, 0)"]

    rx = c * Δt / Δx
    ry = c * Δt / Δy
    r = rx + ry

    if r < 1/√2
        (@warn "CFL stability condition is not met! (r = $r < 1/√2)")
    end

    u = zeros(ny, nx, N)
    u[:,  :, 1] = u0.(xx, yy)  # t = 0, first IC
    u[i,  j, 2] = (0.5 .* (u[i, j, 1] * (2 - 2*(rx^2 + ry^2)) 
                .+ rx^2 * (u[i.+1, j, 1] .+ u[i.-1, j, 1]) 
                .+ ry^2 * (u[i, j.+1, 1] .+ u[i, j.-1, 1])) 
                .+ Δt.*u_t0.(xx, yy)[i, j])  # t = 1, second IC

    u[1,  :, 1:2] .= u_Ay
    u[:,  1, 1:2] .= u_xC
    u[ny, :, 1:2] .= u_By
    u[:, nx, 1:2] .= u_xD

    for t in 3:N # t = 2, 3, ..., T
        u[i, j, t] = (u[i, j, t.-1] * (2 - 2*(rx^2 + ry^2)) 
                    .+ rx^2 * (u[i.+1, j, t.-1] .+ u[i.-1, j, t.-1]) 
                    .+ ry^2 * (u[i, j.+1, t.-1] .+ u[i, j.-1, t.-1]) 
                    .- u[i, j, t.-2])              
    end
    return u
end

explicit_finite_difference (generic function with 1 method)

In [4]:
ICs = Dict(  # Initial Conditions
     "u(x, y, 0)"     => (x, y) -> exp(-x^2 - y^2),
     "∂u/∂t(x, y, 0)" => (x, y) -> 0 #-exp(-x^2 - y^2)
)


BCs = Dict(  # Dirichlet Boundary Conditions
    "u(A, y, t)" => 0,
    "u(B, y, t)" => 0,
    "u(x, C, t)" => 0,
    "u(x, D, t)" => 0,
)

Ω = Dict(  # Domain - rectangle
    "A" => -5,  # x min
    "B" =>  5,  # x max
    "C" => -5,  # y min
    "D" =>  5,  # y max
)


Δx = 0.1 
Δy = 0.1
Δt = 0.1
c  = 0.5
N  = 700

grid_params = grid(Δx=Δx, Δy=Δy, Ω=Ω)
u = explicit_finite_difference(Δx=Δx, Δy=Δy, Δt=Δt, c=c, N=N, ICs=ICs, BCs=BCs, grid_params=grid_params);

In [5]:
function wave_animation(u::Array{Float64, 3}, grid_params::Tuple; plot_size::Tuple=SIZE_1, colormap=COLORMAP_1, title = "", kwargs...)

    frames = size(u)[3]
    (nx, ny, xx, yy) = grid_params
    xs, ys = xx[1, :], yy[:, 1]
    l = @layout [a{0.6w} b]
    camera_angles = LinRange(0, 360, frames)

    an = @animate for t in 1:frames

        p1 = surface(xx, yy, u[:, :, t],
                    zlims=(extrema(u)),
                    colormap=colormap,
                    colorbar=false,
                    zlabel=L"u")

        p2 = heatmap(xs, ys, u[:, :, t], 
                    colormap=colormap)

        plot(p1, p2, 
            layout=l, 
            xlabel=L"x",
            ylabel=L"y",
            clims=extrema(u),
            xlims=extrema(xs),
            ylims=extrema(ys), 
            camera=(camera_angles[t], 30), 
            plot_title= title * "\n\n" * latexstring(" (t = $(round(t*Δt, digits=2)))"),
            aspect_ratio=:equal, 
            size=plot_size,
            link=:all;
            kwargs...)
    end
    return an
end


wave_animation (generic function with 1 method)

In [6]:
an1 = wave_animation(u[:, :, 1:500], grid_params, title = L"u(x, y, 0) = e^{-(x^2 + y^2)}, \quad u_t(x, y, 0) = 0")
gif(an1, "./media/test.gif", fps = 25);

<img src="./media/test.gif">

<div class="text", style="font-family: 'Times New Roman', Times, serif;
                          font-weight: normal;
                          text-align: justify;
                          font-size: 18px;
                          padding: 0px 100px 0px 0px;">
    <h3>References</h3>
    <hr>
[1] &nbsp;<a id="1", href = https://hplgit.github.io/fdm-book/doc/pub/book/sphinx/._book008.html>Center for Biomedical Computing, Finite difference methods for 2D and 3D wave equations.</a>
<br>
[2]&nbsp; <a id="2", href = https://www.google.com/>empty</a>
<br>
[3]&nbsp; <a id="3", href = https://www.google.com/>empty</a>
<br>
[4]&nbsp; <a id="4", href = https://www.google.com/>empty</a>
<br>
[5]&nbsp; <a id="5", href = https://www.google.com/>empty</a>
</div>